In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1: Initial Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
import gc  # For memory management

# Load data with reduced memory usage
def reduce_mem_usage(df):
    """Iterate through dataframe columns and reduce memory usage"""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Initial memory usage: {start_mem:.2f} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {100 * (start_mem - end_mem) / start_mem:.1f}%")
    return df

# Load data
df_lct_dl = pd.read_csv('/kaggle/input/model-data/curr_lct_dl.csv')
df_lct_dl = reduce_mem_usage(df_lct_dl)

# Remove duplicates
print(f"Initial shape: {df_lct_dl.shape}")
df_lct_dl.drop_duplicates(inplace=True)
print(f"After removing duplicates: {df_lct_dl.shape}")

In [ ]:
# This is the most likely correct path
df_lct_dl = pd.read_csv('/kaggle/input/model-data/curr_lct_dl.csv')

In [ ]:
df_lct_dl

In [ ]:
df_lct_dl.info()

In [ ]:
df_lct_dl['error_code'].unique()

In [ ]:
error_code_distribution = df_lct_dl['error_code'].value_counts()
print(error_code_distribution)

In [ ]:
df_lct_dl.describe()

In [ ]:
df_lct_dl[['bytes_sec', 'duration', 'packets_received', 'packets_sent']].describe()

In [ ]:
df_lct_dl.duplicated().value_counts()

In [ ]:
df_lct_dl.isnull().sum()

## Error Code Distribution

- **NO_ERROR**: 866,807 (99.7%)

- **TCP_CONNECT**: 2,336 (0.27%)

- **ZERO_SENT_SERVER**: 382 (0.04%)

- **RCV_FAILED**: 175 (0.02%)

- **TCP_RECV**: 1 (extremely rare)

## Data Characteristics

- **Missing Values**: None

- **Duplicate Rows**: 246 (can be removed)

- **Class Imbalance**: Extremely imbalanced classes (NO_ERROR dominates)

## Numerical Features

- **Variation**: Numerical features exhibit good variation.

- **Predictive Potential**: `bytes_sec` shows high variability and could be highly predictive.

# 2: Feature Engineering and Preprocessing

In [ ]:
# Extract features from datetime
df_lct_dl['dtime'] = pd.to_datetime(df_lct_dl['dtime'])
df_lct_dl['hour'] = df_lct_dl['dtime'].dt.hour.astype(np.int8)
df_lct_dl['day_of_week'] = df_lct_dl['dtime'].dt.dayofweek.astype(np.int8)
df_lct_dl['day_of_month'] = df_lct_dl['dtime'].dt.day.astype(np.int8)
df_lct_dl['month'] = df_lct_dl['dtime'].dt.month.astype(np.int8)

# Drop original datetime column
df_lct_dl.drop('dtime', axis=1, inplace=True)

# Assuming 'error_code' is our target - if not, replace with your actual target
# If target is binary, we'll need to encode it
if 'error_code' in df_lct_dl.columns:
    df_lct_dl['error_code'] = df_lct_dl['error_code'].apply(lambda x: 0 if x == 'NO_ERROR' else 1).astype(np.int8)
    target = 'error_code'
else:
    # Replace with your actual target column
    target = 'target_column'  

# Separate features and target
X = df_lct_dl.drop(target, axis=1)
y = df_lct_dl[target]

# Free up memory
del df_lct_dl
gc.collect()

# Split data - stratify to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['category', 'object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

# 3: Create Preprocessing Pipeline with Memory Efficiency

In [ ]:
# Numerical pipeline with memory-efficient scaler
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline with low-memory one-hot encoding
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=True, dtype=np.int8))
])

# Full preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
], sparse_threshold=0.3)  # Helps with memory for large datasets

# 4: Build Models with Feature Selection

In [ ]:
# SVM Pipeline with reduced memory footprint
svm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(f_classif, k=20)),  # Select top 20 features
    ('classifier', SVC(
        kernel='rbf', 
        class_weight='balanced', 
        probability=True, 
        random_state=42,
        gamma='scale',  # Helps with memory for large datasets
        cache_size=500  # Limit cache size
    ))
])

# Random Forest Pipeline with reduced memory
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(f_classif, k=20)),  # Select top 20 features
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        max_depth=10,  # Limit depth to prevent memory issues
        n_jobs=-1  # Use all cores
    ))
])

# 5: Train and Evaluate Models with Memory Management

In [ ]:
def train_and_evaluate(pipeline, X_train, y_train, X_test, y_test, model_name):
    print(f"\nTraining {model_name}...")
    
    # Fit the model
    pipeline.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = pipeline.predict(X_test)
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Clear memory
    gc.collect()
    return pipeline

# Train SVM (may be memory intensive)
try:
    svm_model = train_and_evaluate(svm_pipeline, X_train, y_train, X_test, y_test, "SVM")
except MemoryError:
    print("SVM training failed due to memory constraints. Trying with smaller sample...")
    # Try with smaller sample if memory is an issue
    X_train_sample, _, y_train_sample, _ = train_test_split(
        X_train, y_train, train_size=0.5, random_state=42, stratify=y_train)
    svm_model = train_and_evaluate(svm_pipeline, X_train_sample, y_train_sample, X_test, y_test, "SVM (50% sample)")

# Train Random Forest (generally more memory-friendly)
rf_model = train_and_evaluate(rf_pipeline, X_train, y_train, X_test, y_test, "Random Forest")

# Compare model performance
print("\nModel training completed!")

# 6: Deployment Preparation

In [ ]:
import joblib
from sklearn.base import BaseEstimator, TransformerMixin

# Create a custom transformer for datetime features
class DateTimeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        if 'dtime' in X.columns:
            X['dtime'] = pd.to_datetime(X['dtime'])
            X['hour'] = X['dtime'].dt.hour.astype(np.int8)
            X['day_of_week'] = X['dtime'].dt.dayofweek.astype(np.int8)
            X['day_of_month'] = X['dtime'].dt.day.astype(np.int8)
            X['month'] = X['dtime'].dt.month.astype(np.int8)
            X.drop('dtime', axis=1, inplace=True)
        return X

# Create a final pipeline that includes datetime processing
final_pipeline = Pipeline([
    ('datetime_processor', DateTimeTransformer()),
    ('preprocessor', preprocessor),
    ('feature_selector', SelectKBest(f_classif, k=20)),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        max_depth=10,
        n_jobs=-1
    ))
])

# Retrain on full data if needed
final_pipeline.fit(X, y)

# Save the model
joblib.dump(final_pipeline, 'network_error_pipeline.pkl', compress=3)  # High compression to save space

print("Pipeline saved for deployment!")

# Visualizing Model Results

# 1. Confusion Matrices

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# 1. First, let's create a more robust preprocessing pipeline
# that will handle feature consistency automatically

# Identify categorical and numerical columns (should be done before any splitting)
categorical_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.tolist()

# Create a robust preprocessing pipeline that maintains feature consistency
preprocessor = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    ), numerical_cols),
    (make_pipeline(
        SimpleImputer(strategy='most_frequent'),
        OneHotEncoder(handle_unknown='infrequent_if_exist', sparse=False)
    ), categorical_cols),
    remainder='drop'  # Drop any columns not explicitly transformed
)

# 2. Rebuild your models with this consistent preprocessing
svm_pipeline = make_pipeline(
    preprocessor,
    SelectKBest(f_classif, k=20),
    SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
)

rf_pipeline = make_pipeline(
    preprocessor,
    SelectKBest(f_classif, k=20),
    RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, max_depth=10)
)

# 3. Retrain the models
print("Retraining models with consistent preprocessing...")
svm_model = svm_pipeline.fit(X_train, y_train)
rf_model = rf_pipeline.fit(X_train, y_train)

# 4. Now the predictions should work correctly
def plot_confusion_matrix(y_true, y_pred, model_name, ax=None):
    cm = confusion_matrix(y_true, y_pred)
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['No Error', 'Error'],
                yticklabels=['No Error', 'Error'])
    ax.set_title(f'{model_name} Confusion Matrix')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

# Get predictions from both models
try:
    y_pred_svm = svm_model.predict(X_test)
    y_pred_rf = rf_model.predict(X_test)
    
    # Create side-by-side confusion matrices
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    plot_confusion_matrix(y_test, y_pred_svm, 'SVM', ax1)
    plot_confusion_matrix(y_test, y_pred_rf, 'Random Forest', ax2)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error during prediction: {e}")
    print("\nDebugging information:")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print("Checking column differences:")
    train_cols = set(X_train.columns)
    test_cols = set(X_test.columns)
    print(f"Columns in train but not test: {train_cols - test_cols}")
    print(f"Columns in test but not train: {test_cols - train_cols}")

# 2. ROC Curves and AUC Scores

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

def plot_roc_curve(y_true, y_probs, model_name, ax=None):
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, 
                   estimator_name=model_name).plot(ax=ax)
    ax.plot([0, 1], [0, 1], 'k--')  # Diagonal line
    ax.set_title(f'ROC Curve - {model_name}')
    return roc_auc

# Get probability predictions (we need to use predict_proba)
y_proba_svm = svm_model.predict_proba(X_test)[:, 1]
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Plot ROC curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
auc_svm = plot_roc_curve(y_test, y_proba_svm, 'SVM', ax1)
auc_rf = plot_roc_curve(y_test, y_proba_rf, 'Random Forest', ax2)
plt.tight_layout()
plt.show()

print(f"SVM AUC: {auc_svm:.4f}")
print(f"Random Forest AUC: {auc_rf:.4f}")

# 3. Precision-Recall Curves

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

def plot_pr_curve(y_true, y_probs, model_name, ax=None):
    precision, recall, _ = precision_recall_curve(y_true, y_probs)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    PrecisionRecallDisplay(precision=precision, recall=recall, 
                         estimator_name=model_name).plot(ax=ax)
    ax.set_title(f'Precision-Recall Curve - {model_name}')
    return auc(recall, precision)

# Plot PR curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
pr_auc_svm = plot_pr_curve(y_test, y_proba_svm, 'SVM', ax1)
pr_auc_rf = plot_pr_curve(y_test, y_proba_rf, 'Random Forest', ax2)
plt.tight_layout()
plt.show()

print(f"SVM PR AUC: {pr_auc_svm:.4f}")
print(f"Random Forest PR AUC: {pr_auc_rf:.4f}")